In [21]:
import mxnet as mx
import mxnet.gluon.nn as mgnn
import numpy as np
from mxnet import gluon
from mxnet import autograd

In [39]:
class FM(mgnn.Block):
    def __init__(self, hidden_dim, user_dim, item_dim, **kwargs):
        super(FM, self).__init__(**kwargs)
        self.hidden_dim = hidden_dim
        self.user_dim = user_dim
        self.item_dim = item_dim
        with self.name_scope():
            self.embedding_user = mgnn.Embedding(input_dim=self.user_dim, output_dim=self.hidden_dim)
            self.embedding_item = mgnn.Embedding(input_dim=self.item_dim, output_dim=self.hidden_dim)


    def forward(self, x):
        """

        :param x:
        :type x: mx.ndarray
        :return:
        """
        user = x[:, 0]
        item = x[:, 1]
        emb_user = mx.ndarray.flatten(self.embedding_user(user))
        emb_item = mx.ndarray.flatten(self.embedding_item(item))
        r = mx.nd.sum(emb_user * emb_item, axis=1)
        return r

    def print_params(self):
        print(self.collect_params())
        print(self.name_scope())
        print(self.prefix)

In [40]:
new_data = np.array([
    [0, 0],
    [0, 1],
    [0, 2],
    [1, 0],
    [1, 1],
    [1, 2],
    [2, 0],
    [2, 1],
    [2, 2]
])
label_data = np.array([1, 2, 3, 2, 0, 3, 3, 3, 0])

In [41]:
net = FM(hidden_dim=2, user_dim=3, item_dim=3, prefix="ewlfm_")

In [42]:
net.collect_params()

ewlfm_ (
  Parameter ewlfm_embedding0_weight (shape=(3, 2), dtype=None)
  Parameter ewlfm_embedding1_weight (shape=(3, 2), dtype=None)
)

In [43]:
net.initialize()

In [44]:
net.forward(mx.nd.array(new_data))


[ 0.00244576  0.00137699  0.00096017  0.00309144  0.00148894  0.00088894
  0.00291376  0.00137204  0.00079742]
<NDArray 9 @cpu(0)>

In [45]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.7})

In [46]:
l2_loss = gluon.loss.L2Loss()

In [47]:
for epoch in range(10):
    train_loss = 0.
    train_acc = 0.
    data = mx.nd.array(new_data)
    label = mx.nd.array(label_data)
    with autograd.record():
        output = net(data)
        loss = l2_loss(output, label)
    loss.backward()
    trainer.step(label_data.shape[0])
    print(mx.nd.sum(loss).asscalar())
    #     with mx.autograd.record():
    #         output = net(data)
    #         loss = l2_loss(output, label)
    #     loss.backward()
    #     trainer.step(batch_size)
    # 
    #     train_loss += nd.mean(loss).asscalar()
    #     train_acc += utils.accuracy(output, label)
    # 
    # test_acc = utils.evaluate_accuracy(test_data, net)
    # print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
    #     epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

22.4702
22.4328
22.3595
22.2095
21.903
21.2857
20.0834
17.8984
14.4478
10.3046


In [25]:
label_data.shape[0]

9

In [49]:
class FM2(mgnn.Block):
    
    def __init__(self, hidden_dim, in_units, **kwargs):
        super(FM2, self).__init__(**kwargs)
        self.hidden_dim = hidden_dim
        with self.name_scope():
            self.w = self.params.get('w', shape=(in_units, 1))
            self.b = self.params.get('b', shape=(1,))
            self.v = self.params.get('v', shape=(in_units, hidden_dim))
            pass
    
    def forward(self, x):
        net_1 = mx.ndarray.dot(x, self.w) + self.b
        xv = mx.nd.dot(x, self.v)
        net_2 = mx.nd.sum(xv * xv, axis=1)
        net_3 = mx.nd.sigmoid(net_1 + net_2)
        return net_3

In [53]:
a = mx.nd.array([[1, 2, 3], [4, 5, 6]])

In [54]:
a


[[ 1.  2.  3.]
 [ 4.  5.  6.]]
<NDArray 2x3 @cpu(0)>

In [55]:
v = mx.nd.array([[1, 2], [3, 3], [2, 6]])

In [57]:
xv = mx.nd.dot(a, v)

In [58]:
mx.nd.dot(xv, xv.transpose())


[[  845.  1937.]
 [ 1937.  4442.]]
<NDArray 2x2 @cpu(0)>

In [60]:
mx.nd.sum(xv * xv, axis=1)


[  845.  4442.]
<NDArray 2 @cpu(0)>